# Traffic Vehicle Counter

## Project Overview
A Python-based traffic vehicle detection and counting system using **YOLOv8** object detection model.

### Features
- Real-time vehicle detection using YOLOv8
- Video processing with object tracking
- Vehicle counting and **lane detection**
- Output annotated video with detection results
- **Lane-specific counting** (left or right)

### Supported Vehicle Classes
- Cars
- Bicycles
- Motorcycles
- Trucks

## 1. Install Requirements

First, install all required dependencies:

In [ ]:
# Install required packages
import subprocess
import sys

requirements = [
    'ultralytics==8.0.0',
    'opencv-python==4.8.0.74',
    'torch==2.0.0',
    'torchvision==0.15.0',
    'numpy==1.24.3',
    'pillow==10.0.0',
    'pyyaml==6.0',
    'requests==2.31.0'
]

for package in requirements:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])

print("✅ All requirements installed successfully!")

## 2. Import Required Libraries

In [ ]:
import cv2
import os
import numpy as np
from ultralytics import YOLO
from collections import defaultdict

print("✅ All libraries imported successfully!")

## 3. Define Vehicle Classes and Configuration

In [ ]:
# Allowed vehicle classes (COCO dataset IDs)
VEHICLE_CLASSES = {
    2: "car",
    1: "bicycle",
    3: "motorcycle",
    7: "truck"
}

print("Vehicle Classes:")
for cls_id, name in VEHICLE_CLASSES.items():
    print(f"  {cls_id}: {name}")

## 4. Define Vehicle Detection and Counting Function

In [ ]:
def process_video(input_path, output_path, lane_side="left"):
    """
    Process video file for vehicle detection and counting.
    
    Parameters:
    -----------
    input_path : str
        Path to input video file
    output_path : str
        Path to save output video with detections
    lane_side : str
        Lane to count vehicles from ("left" or "right")
    """
    
    # Ensure models folder exists
    os.makedirs("models", exist_ok=True)

    model_path = "models/yolov8n.pt"

    # Load YOLO model (downloads automatically first time)
    print(f"Loading YOLO model from {model_path}...")
    model = YOLO(model_path)

    # Open video capture
    cap = cv2.VideoCapture(input_path)

    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"Video Properties:")
    print(f"  Resolution: {width}x{height}")
    print(f"  FPS: {fps}")
    print(f"  Total Frames: {total_frames}")
    print(f"  Lane Side: {lane_side}")

    # Ensure output directory exists
    os.makedirs(os.path.dirname(output_path) or ".", exist_ok=True)

    # Create video writer
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Track vehicle counts
    class_counts = defaultdict(int)
    counted_ids = set()

    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % 30 == 0:
            print(f"Processing frame {frame_count}/{total_frames}...")

        # Run YOLOv8 tracking
        results = model.track(frame, persist=True, verbose=False)

        # Calculate mid-point for lane division
        mid_x = width // 2

        # Draw vertical lane divider
        cv2.line(frame, (mid_x, 0), (mid_x, height), (0, 255, 255), 2)

        if results[0].boxes.id is not None:

            boxes = results[0].boxes.xyxy.cpu().numpy()
            class_ids = results[0].boxes.cls.cpu().numpy().astype(int)
            ids = results[0].boxes.id.cpu().numpy().astype(int)

            for box, cls_id, obj_id in zip(boxes, class_ids, ids):

                # Skip non-vehicle classes
                if cls_id not in VEHICLE_CLASSES:
                    continue

                x1, y1, x2, y2 = map(int, box)
                center_x = (x1 + x2) // 2

                # Lane filtering
                if lane_side == "left" and center_x > mid_x:
                    continue
                if lane_side == "right" and center_x < mid_x:
                    continue

                label = VEHICLE_CLASSES[cls_id]

                # Count only once per vehicle ID
                if obj_id not in counted_ids:
                    class_counts[label] += 1
                    counted_ids.add(obj_id)

                # Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                # Show ID + Class
                text = f"ID:{obj_id} {label}"
                cv2.putText(frame, text, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.6, (0, 255, 0), 2)

        # Display Vehicle Counts
        car = class_counts["car"]
        bicycle = class_counts["bicycle"]
        motorcycle = class_counts["motorcycle"]
        truck = class_counts["truck"]
        total = car + bicycle + motorcycle + truck

        cv2.putText(frame, f"Car: {car}", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

        cv2.putText(frame, f"Bicycle: {bicycle}", (20, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

        cv2.putText(frame, f"Motorcycle: {motorcycle}", (20, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

        cv2.putText(frame, f"Truck: {truck}", (20, 130),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

        cv2.putText(frame, f"Total: {total}", (20, 170),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        out.write(frame)

    cap.release()
    out.release()
    
    print("\n" + "="*50)
    print("✅ Processing completed successfully!")
    print("="*50)
    print(f"Total Vehicles Counted ({lane_side} lane):")
    print(f"  Cars: {class_counts['car']}")
    print(f"  Bicycles: {class_counts['bicycle']}")
    print(f"  Motorcycles: {class_counts['motorcycle']}")
    print(f"  Trucks: {class_counts['truck']}")
    print(f"  ────────────────")
    print(f"  TOTAL: {sum(class_counts.values())}")
    print("="*50)
    print(f"Output saved to: {output_path}")

print("✅ process_video function defined successfully!")

## 5. Configuration - Set Input and Output Paths

In [ ]:
# Configuration
input_path = "data/video/raw/Traffic_video_demo.mp4"
output_path = "data/processed/output.mp4"

# Change to "left" or "right" for lane-specific counting
lane_side = "left"

print("Configuration:")
print(f"  Input Path: {input_path}")
print(f"  Output Path: {output_path}")
print(f"  Lane Side: {lane_side}")
print(f"  Input file exists: {os.path.exists(input_path)}")

## 6. Run Vehicle Detection and Counting

Execute the video processing. This will:
1. Load the YOLOv8 model
2. Process each frame of the video
3. Detect and track vehicles
4. Count vehicles by type in the specified lane
5. Save annotated video with bounding boxes and counts

In [ ]:
# Run the video processing
if os.path.exists(input_path):
    process_video(input_path, output_path, lane_side)
else:
    print(f"❌ Error: Input video not found at {input_path}")
    print("Please ensure the video file exists in the data/video/raw/ directory")

## 7. Alternative: Process Multiple Videos

You can also process multiple videos in a loop:

In [ ]:
# Example: Process multiple videos

# videos_to_process = [
#     ("data/video/raw/video1.mp4", "data/processed/output1.mp4", "left"),
#     ("data/video/raw/video2.mp4", "data/processed/output2.mp4", "right"),
# ]
# 
# for input_vid, output_vid, lane in videos_to_process:
#     if os.path.exists(input_vid):
#         print(f"\nProcessing {input_vid}...")
#         process_video(input_vid, output_vid, lane)
#     else:
#         print(f"Skipping {input_vid} - file not found")

print("Uncomment the code above to process multiple videos")

## Results Summary

After running the vehicle detection:
- The output video will be saved with:
  - **Green bounding boxes** around detected vehicles
  - **Vehicle IDs** and class labels
  - **Yellow vertical line** dividing the lanes
  - **Vehicle counts** displayed in the top-left corner

### Output Information
- **Path:** Check the `output_path` variable
- **Format:** MP4 video
- **Resolution:** Same as input video
- **FPS:** Same as input video

## Project Structure

```
traffic-vehicle-counter/
├── models/
│   └── yolov8n.pt                    # Pre-trained YOLOv8 nano model
├── data/
│   └── video/
│       ├── raw/                      # Input video files
│       │   └── Traffic_video_demo.mp4
│       └── processed/                # Output video files
│           └── output.mp4
├── main.py                           # Original main script
├── detection.py                      # Detection functions
├── Traffic_Vehicle_Counter.ipynb     # This notebook
├── requirements.txt                  # Project dependencies
└── README.md                         # Project documentation
```